<a href="https://colab.research.google.com/github/ramaoutlook/fakeNewsClassificationModel/blob/main/Project4_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imagenet dataset. imagenet website is for images dataset for classification or machine learning practice.

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [4]:
# For True.csv
true = pd.read_csv('/content/True.csv', engine='python', on_bad_lines='skip')


In [5]:
# For Fake.csv
fake = pd.read_csv('/content/Fake.csv', engine='python', on_bad_lines='skip')

In [6]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
true['label'] = 1
fake['label'] = 0

In [9]:
true.head(2)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1


In [10]:
news = pd.concat([fake, true], axis =0)

In [11]:
news.head(2)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0


In [12]:
news.tail(2)

,title,text,subject,date,label
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [13]:
news.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [14]:
news.drop(['title', 'subject', 'date'], axis = 1, inplace = True)

In [15]:
news.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [16]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
 4   label    21417 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 836.7+ KB


In [17]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   label    23481 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 917.4+ KB


In [18]:
news = news.sample(frac=1)   #Refhuffling
news.head()

,text,label
6544,There has been a national uproar over HB2 ever...,0
19384,BANGKOK (Reuters) - A roadside bomb planted by...,1
21774,So words deemed offensive are only offensive i...,0
899,Rep. Adam Schiff (D-CA) is the Ranking Member ...,0
8320,(Reuters) - New Jersey Governor Chris Christie...,1


In [19]:
news.columns

Index(['text', 'label'], dtype='object')

In [20]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 6544 to 7664
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   label   44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [21]:
news.head()

,text,label
6544,There has been a national uproar over HB2 ever...,0
19384,BANGKOK (Reuters) - A roadside bomb planted by...,1
21774,So words deemed offensive are only offensive i...,0
899,Rep. Adam Schiff (D-CA) is the Ranking Member ...,0
8320,(Reuters) - New Jersey Governor Chris Christie...,1


In [22]:
news.columns

Index(['text', 'label'], dtype='object')

In [23]:
news.columns[0]

'text'

In [24]:
news.reset_index()

,index,text,label
0,6544,There has been a national uproar over HB2 ever...,0
1,19384,BANGKOK (Reuters) - A roadside bomb planted by...,1
2,21774,So words deemed offensive are only offensive i...,0
3,899,Rep. Adam Schiff (D-CA) is the Ranking Member ...,0
4,8320,(Reuters) - New Jersey Governor Chris Christie...,1
...,...,...,...
44893,15686,BEIRUT (Reuters) - Iran put a ballistic missil...,1
44894,20092,This new information just adds more validity T...,0
44895,8830,"Almost 20 years ago, David Bowie released a mu...",0
44896,1290,WASHINGTON (Reuters) - A bipartisan pair of Mi...,1


In [25]:
news.columns

Index(['text', 'label'], dtype='object')

In [26]:
import re

In [27]:
# # prompt: give me code to remove any urls using regex and puctuation and digits and newline characters direclty with re.sub not re.compile. and any other data cleaning code.

# # Function to clean the text
# def clean_text(text):
#     # Remove URLs
#     text = re.sub(r'http\S+', '', text)
#     # Remove punctuation and digits
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     # Remove newline characters
#     text = re.sub(r'\n', '', text)
#     # Convert to lowercase
#     text = text.lower()
#     return text

# # Apply the cleaning function to the 'text' column
# news['text'] = news['text'].apply(clean_text)

# # Further cleaning: remove stopwords and stemming (optional but common)
# ps = PorterStemmer()
# corpus = []
# for i in range(len(news)):
#   review = re.sub('[^a-zA-Z]', ' ', news['text'].iloc[i])
#   review = review.lower()
#   review = review.split()
#   review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#   review = ' '.join(review)
#   corpus.append(review)

# # Add the cleaned text as a new column
# news['cleaned_text'] = corpus

# # Display the first few rows with the cleaned text
# print(news.head())

# # Example of further text processing: Vectorization
# # cv = CountVectorizer(max_features=5000)
# # X = cv.fit_transform(corpus).toarray()
# # y = news['label']
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


In [28]:
def wordopt(text):
  #Convert into lowercase
  text = text.lower()

  # Remove URLs
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  #Remoeve tags
  text = re.sub(r'.*?>', '', text)
  #Remove punctuation
  text = re.sub(r'[^\w\s]', '', text)
  #Remove digits
  text = re.sub(r'\d+', '', text)
  #Remove newline characters
  text = re.sub(r'\n', '', text)
  # #Remove stopwords
  # stop_words = set(stopwords.words('english'))
  # text = ' '.join([word for word in text.split() if word not in stop_words])
  # #Stemming
  # ps = PorterStemmer()
  # text = ' '.join([ps.stem(word) for word in text.split()])

  text = re.sub('[^a-zA-Z]', ' ', text)

  return text

In [31]:
# prompt: from news take 0s 2000 rows and 1s 2000 rows only and apply the code news['text'] = news['text'].apply(wordopt)

import pandas as pd
# Separate news by label
news_0 = news[news['label'] == 0].sample(n=500, random_state=42)
news_1 = news[news['label'] == 1].sample(n=500, random_state=42)

# Concatenate the selected rows
news_subset = pd.concat([news_0, news_1], axis=0)

# Apply the wordopt function
news_subset['text'] = news_subset['text'].apply(wordopt)

# Display the first few rows of the subset with the processed text
print(news_subset.head())

# You can continue processing with news_subset
# For example, if you want to reset the index:
news_subset = news_subset.reset_index(drop=True)
print(news_subset.head())

                                                    text  label
16996  unreal mooch gets millions for pakistani girls...      0
20090  for anyone who s paying attention hillary just...      0
3009   admit it ever since news broke that donald tru...      0
10383  low blow political hack angela rye refused to ...      0
6040   republican house speaker paul ryan has been ge...      0
                                                text  label
0  unreal mooch gets millions for pakistani girls...      0
1  for anyone who s paying attention hillary just...      0
2  admit it ever since news broke that donald tru...      0
3  low blow political hack angela rye refused to ...      0
4  republican house speaker paul ryan has been ge...      0


In [29]:
# news['text'] = news['text'].apply(wordopt)
# news['text']

KeyboardInterrupt: 

In [35]:
x = news_subset['text']
y = news_subset['label']

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [37]:
x_train.shape

(700,)

In [38]:
x_test.shape

(300,)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [43]:
vectorization = TfidfVectorizer()

In [44]:
xv_train = vectorization.fit_transform(x_train)

In [45]:
xv_test = vectorization.transform(x_test)

In [46]:
xv_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 148143 stored elements and shape (700, 21662)>

In [47]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 53367 stored elements and shape (300, 21662)>

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
LR = LogisticRegression()

In [52]:
LR.fit(xv_train, y_train)

LogisticRegression()

In [53]:
pred_lr = LR.predict(xv_test)

In [54]:
LR.score(xv_test, y_test)

0.9566666666666667

In [63]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [64]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       144
           1       0.95      0.97      0.96       156

    accuracy                           0.96       300
   macro avg       0.96      0.96      0.96       300
weighted avg       0.96      0.96      0.96       300



In [55]:
print(confusion_matrix(y_test, pred_lr))

[[136   8]
 [  5 151]]


In [56]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier()

In [57]:
DTC.fit(xv_train, y_train)

DecisionTreeClassifier()

In [58]:
pred_dtc = DTC.predict(xv_test)

In [59]:
DTC.score(xv_test, y_test)

0.97

In [65]:
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       144
           1       0.99      0.96      0.97       156

    accuracy                           0.97       300
   macro avg       0.97      0.97      0.97       300
weighted avg       0.97      0.97      0.97       300



In [68]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)

In [69]:
rfc.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [70]:
predict_rfc = rfc.predict(xv_test)

In [71]:
rfc.score(xv_test, y_test)

0.9633333333333334

In [72]:
print(classification_report(y_test, predict_rfc))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       144
           1       0.96      0.97      0.96       156

    accuracy                           0.96       300
   macro avg       0.96      0.96      0.96       300
weighted avg       0.96      0.96      0.96       300



In [73]:
from sklearn.ensemble import GradientBoostingClassifier


In [74]:
gbc = GradientBoostingClassifier(random_state=0)

In [75]:
gbc.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [76]:
predict_gbc = gbc.predict(xv_test)

In [77]:
gbc.score(xv_test, y_test)

0.9833333333333333

In [78]:
print(classification_report(y_test, predict_gbc))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       144
           1       0.97      0.99      0.98       156

    accuracy                           0.98       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300



In [79]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Genuine News"

In [80]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_gbc = gbc.predict(new_xv_test)
    pred_rfc = rfc.predict(new_xv_test)
    pred_dtc = DTC.predict(new_xv_test)
    return "\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {} \nDTC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                         output_lable(pred_gbc[0]),
                                                                                                         output_lable(pred_rfc[0]),
                                                                                                         output_lable(pred_dtc[0]))

In [81]:
news_article = str(input())

Rourkela: a 42-year-old man was killed by his firend for all... read mofe at: https:://google.com


In [82]:
manual_testing(news_article)

'\n\nLR Prediction: Fake News \nGBC Prediction: Fake News \nRFC Prediction: Fake News \nDTC Prediction: Fake News'

In [83]:
news_article = str(input())


modi dived from everest with amit shah and sonia gandhi and priyanka gandhi. rahul gandhi put a sword in the below where modi is going to jump so that he will be killed


In [84]:
manual_testing(news_article)

'\n\nLR Prediction: Fake News \nGBC Prediction: Fake News \nRFC Prediction: Fake News \nDTC Prediction: Fake News'

In [85]:
news_article = str(input())


election results are out and bjp won the election and modi is again the prime minister for 3rd time


In [86]:
manual_testing(news_article)

'\n\nLR Prediction: Fake News \nGBC Prediction: Fake News \nRFC Prediction: Fake News \nDTC Prediction: Fake News'

In [87]:
news_article = str(input())


donald trump won the election


In [88]:
manual_testing(news_article)

'\n\nLR Prediction: Fake News \nGBC Prediction: Fake News \nRFC Prediction: Fake News \nDTC Prediction: Fake News'